**Read data from .csv files in the form of pandas dataframes**

In [ ]:
import pandas as pd

train_data_df = pd.read_csv('../input/train.csv')
test_data_df = pd.read_csv('../input/test.csv')

**Extract training and testing data from the dataframes above**

In [ ]:
train_data = train_data_df.values
test_data = test_data_df.values

train_features = train_data[:, 3:25][0:3485869]
train_targets = train_data[:, 25][0:3485869]

val_features = train_data[:, 3:25][3485869:4357336]
val_targets = train_data[:, 25][3485869:4357336]

test_features = test_data[:, 3:25]
test_data = test_data[:, 0:3]

In [ ]:
import sklearn

**Scale all features into a range of (-1, 1)**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1)).fit(train_features)

train_features = scaler.transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

**Build and train Bagging Regression model...**

In [ ]:
from sklearn.ensemble import BaggingRegressor

model = BaggingRegressor()
model.fit(train_features, train_targets)

**Predict winPlacePerc values for test data using the trained Bagging model**

In [ ]:
predictions = model.predict(test_features)
test_data_df['winPlacePercPred'] = predictions

**Rank the groups in the each PUBG game based on average predicted winPlacePerc values of each group :**

In [ ]:
test_data_preds = test_data_df.groupby(['matchId','groupId'])['winPlacePercPred'].agg('mean').groupby('matchId').rank(pct=True).reset_index()

**Attach the new ranking-based predictions to test dataframe**

In [ ]:
test_data_df['winPlacePerc'] = test_data_preds['winPlacePercPred']

In [ ]:
import numpy as np

**Prepare the final submission file**

In [ ]:
predictions = pd.DataFrame(np.transpose(np.array([test_data[:, 0], list(predictions)])))
predictions.columns = ['Id', 'winPlacePerc']
predictions['Id'] = np.int32(predictions['Id'])
predictions.to_csv('PUBG_preds2.csv', index=False)

Courtesy to [Misha Lisovyi](https://www.kaggle.com/mlisovyi/relativerank-of-predictions) for giving me the idea to use ranking  of groups in each match.